In [96]:
import pandas as pd

In [97]:
df2019 = pd.read_csv('dataset/2019-plu-total-hab-data.csv')
df2020 = pd.read_csv('dataset/2020-plu-total-hab-data.csv')
df2021 = pd.read_csv('dataset/2021-plu-total-hab-data.csv')
df2022 = pd.read_csv('dataset/2022-plu-total-hab-data.csv')
dataframes = [df2019, df2020, df2021, df2022]

In [98]:
# check if all df's have same columns and all columns are the same dataType
if all([all(dataframes[0].columns.sort_values() == df.columns.sort_values()) for df in dataframes]):
    print(all([all(dataframes[0].dtypes.sort_values() == df.dtypes.sort_values()) for df in dataframes]))

True


In [99]:
# check if anything to take note of before merging df's
display(df2019.columns)
display(df2019['Current Year Week Ending'].head()) # check if year is included before merging

Index(['Geography', 'Timeframe', 'Current Year Week Ending', 'Type',
       'ASP Current Year', 'Total Bulk and Bags Units', '4046 Units',
       '4225 Units', '4770 Units', 'TotalBagged Units', 'SmlBagged Units',
       'LrgBagged Units', 'X-LrgBagged Units', 'Bulk GTIN'],
      dtype='object')

0    2019-01-07 00:00:00
1    2019-01-13 00:00:00
2    2019-01-20 00:00:00
3    2019-01-27 00:00:00
4    2019-02-03 00:00:00
Name: Current Year Week Ending, dtype: object

All the datframes have the same columns and all columns have the same datatype so no changes are required there before we merge the columns.

In [100]:
df = pd.concat(dataframes, ignore_index=True)
df.nunique()

Geography                       60
Timeframe                        1
Current Year Week Ending       205
Type                             2
ASP Current Year             24190
Total Bulk and Bags Units    24170
4046 Units                   23004
4225 Units                   22836
4770 Units                   14068
TotalBagged Units            24123
SmlBagged Units                  1
LrgBagged Units                  1
X-LrgBagged Units                1
Bulk GTIN                    23313
dtype: int64

In [101]:
# record columns to delete (ones with only 1 unique value) as not useful for EDA
to_del = df.columns[df.nunique() == 1]

[display(f'{i}: {df[i].unique()}') for i in to_del]
df.drop(to_del, axis=1, inplace=True);

"Timeframe: ['Weekly' nan]"

'SmlBagged Units: [ 0. nan]'

'LrgBagged Units: [ 0. nan]'

'X-LrgBagged Units: [ 0. nan]'

In [102]:
df_num = df.select_dtypes(include='float64') # all numerical features are float64
df_num = df_num / df_num.mean()
df_num.var() # if variance of feature too low, might be better to remove.

ASP Current Year              0.074365
Total Bulk and Bags Units    16.665804
4046 Units                   19.190016
4225 Units                   19.595895
4770 Units                   23.452120
TotalBagged Units            15.136727
Bulk GTIN                    17.219243
dtype: float64

In [103]:
# deal with Na entries
display(df[df.isna().any(axis=1)])
df.dropna(inplace=True)

,Geography,Current Year Week Ending,Type,ASP Current Year,Total Bulk and Bags Units,4046 Units,4225 Units,4770 Units,TotalBagged Units,Bulk GTIN
6136,Source: IRI confidential information. HAB's ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12273,Source: IRI confidential information. HAB's ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18410,Source: IRI confidential information. HAB's ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24193,Source: IRI confidential information. HAB's ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# converting the float values of Total Bulk and Bags Units, 4046 Units, 4225 Units, 4770 Units, Bulk GTIN and TotalBagged Units to integers by rounding them. 
round_features = ['Total Bulk and Bags Units', '4046 Units', '4225 Units', '4770 Units', 'TotalBagged Units', 'Bulk GTIN']
for feature in round_features:
    df[feature] = df[feature].apply(lambda x: round(x))

In [105]:
# take out timing of current Year week ending (00:00:00)
def TakeOutTiming(weekEnding: str):
    return weekEnding[:-8]
df['Current Year Week Ending'] = df['Current Year Week Ending'].apply(TakeOutTiming)

In [106]:
df.to_csv('dataset/avocado_cleaned.csv')